# Part 1 (SQL): Submission 1

In [ ]:
-- 1. Retrieve the total confirmed, death, and recovered cases.

SELECT
    SUM("Confirmed") AS total_confirmed,
    SUM("Deaths") AS total_deaths,
    SUM("Recovered") AS total_recovered
FROM
    covid_data;
    
    

-- 2. Retrieve the total confirmed, deaths and recovered cases for the first quarter
-- of each year of observation.

SELECT
    EXTRACT(YEAR FROM "ObservationDate") AS observation_year,
    SUM("Confirmed") AS total_confirmed,
    SUM("Deaths") AS total_deaths,
    SUM("Recovered") AS total_recovered
FROM
    covid_data
WHERE
    EXTRACT(MONTH FROM "ObservationDate") BETWEEN 1 AND 3
GROUP BY
    observation_year;
    
    
    
-- 3. Retrieve a summary of all the records.

SELECT
    "Country",
    SUM("Confirmed") AS total_confirmed,
    SUM("Deaths") AS total_deaths,
    SUM("Recovered") AS total_recovered
FROM
    covid_data
GROUP BY
    "Country";
    
    

-- 4. Retrieve the percentage increase in the number of death cases from 2019 to 2020.

SELECT
    ((SUM("Deaths") FILTER (WHERE EXTRACT(YEAR FROM "ObservationDate") = 2020)
    - SUM("Deaths") FILTER (WHERE EXTRACT(YEAR FROM "ObservationDate") = 2019))
    ::numeric / SUM("Deaths") FILTER (WHERE EXTRACT(YEAR FROM "ObservationDate") = 2019)) * 100 AS percentage_increase
FROM
    covid_data
WHERE
    EXTRACT(YEAR FROM "ObservationDate") IN (2019, 2020);

    

-- 5. Retrieve information for the top 5 countries with the highest confirmed cases.

SELECT
    "Country",
    SUM("Confirmed") AS total_confirmed
FROM
    covid_data
GROUP BY
    "Country"
ORDER BY
    total_confirmed DESC
LIMIT 5;


-- 6. Compute the total number of drop (decrease) or increase in the confirmed cases from month to month
-- in the 2 years of observation.
WITH monthly_changes AS (
    SELECT
        EXTRACT(YEAR FROM "ObservationDate") AS observation_year,
        EXTRACT(MONTH FROM "ObservationDate") AS observation_month,
        SUM("Confirmed") - LAG(SUM("Confirmed")) OVER (ORDER BY EXTRACT(YEAR FROM "ObservationDate"), EXTRACT(MONTH FROM "ObservationDate")) AS confirmed_change
    FROM
        covid_data
    WHERE
        EXTRACT(YEAR FROM "ObservationDate") IN (2019, 2020)
    GROUP BY
        observation_year, observation_month
    ORDER BY
        observation_year, observation_month
)
SELECT
    observation_year,
    observation_month,
    confirmed_change,
    CASE
        WHEN confirmed_change > 0 THEN 'Increase'
        WHEN confirmed_change < 0 THEN 'Decrease'
        ELSE 'No Change'
    END AS change_type
FROM
    monthly_changes;
